In [80]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import numpy as np
import pandas as pd
from google.colab import files
from pandas_gbq import to_gbq


client = bigquery.Client(project='database-project-431312')

In [81]:
# Tables criation
# Contract
create_table_contract = """
CREATE TABLE `database-project-431312.clients.Contract` (
    Id INT64 NOT NULL,
    ProjectName STRING,
    ContractNum STRING,
    Outstanding FLOAT64
)
"""

query_job = client.query(create_table_contract)
query_job.result()
print("Table created successfully.")


Table created successfully.


In [83]:
# ContractClient
create_table_contract_client = """
CREATE TABLE `database-project-431312.clients.ContractClient` (
    Id INT64 NOT NULL,
    ContractId INT64,
    ClientId INT64,
    ClientType STRING
)
"""

query_job = client.query(create_table_contract_client)
query_job.result()

print("Table created successfully.")

Table created successfully.


In [84]:
# Client
create_table_client = """
CREATE TABLE `database-project-431312.clients.Client` (
    Id INT64 NOT NULL,
    Surname STRING,
    Name STRING,
    SecondName STRING,
    IdentificationNumber STRING
)
"""

query_job = client.query(create_table_client)
query_job.result()

print("Table created successfully.")

Table created successfully.


In [85]:
# ClientProperty
create_table_client_property = """
CREATE TABLE `database-project-431312.clients.ClientProperty` (
    Id INT64 NOT NULL,
    ClientId INT64,
    PropertyId INT64
)
"""

query_job = client.query(create_table_client_property)
query_job.result()

In [88]:
# Property
create_table_property = """
CREATE TABLE `database-project-431312.clients.Property` (
    Id INT64 NOT NULL,
    Region STRING,
    PropertyType STRING,
    PropertyArea FLOAT64,
    PropertyPrice FLOAT64
)
"""

query_job = client.query(create_table_property)
query_job.result()


In [90]:
# Payments
create_table_payments = """
CREATE TABLE `database-project-431312.clients.Payments` (
    Id INT64 NOT NULL,
    ContractId INT64,
    PaymentDate DATE,
    PaymentSum FLOAT64
)
"""

query_job = client.query(create_table_payments)
query_job.result()


In [91]:
# Uploading an Excel file from a local computer
uploaded = files.upload()

Saving clients_to_upload.xlsx to clients_to_upload (1).xlsx


In [92]:
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name, sheet_name='Sheet1')

In [93]:
df.head()


,Проект,Номер кредитного договору,"Сума боргу, грн",Тип клієнта,ПІБ,ІПН,"Регіон майна, що значиться за клієнтом",Тип майна клієнта,Площа майна клієнта,"Оцінка майна клієнта, дол"
0,Дельта,6/8133/3007-840,"2050,51",боржник,Іваненко Петро Михайлович,2049702311,Харківська область,будинок,"193,24",20 000
1,Дельта,6/8133/3007-840,"2050,51",боржник,Іваненко Петро Михайлович,2049702311,Харківська область,земля,512,5 000
2,Дельта,12876619000,"30000,03",боржник,Петренко Михайло Іванович,2089702343,Полтавська область,квартира,"57,2",31 500
3,Дельта,12876619000,"30000,03",фінансовий поручитель,Михайленко Іванна Петрівна,2589712664,NaN,NaN,NaN,NaN


In [94]:
df.dtypes

,0
Проект,object
Номер кредитного договору,object
"Сума боргу, грн",object
Тип клієнта,object
ПІБ,object
ІПН,int64
"Регіон майна, що значиться за клієнтом",object
Тип майна клієнта,object
Площа майна клієнта,object
"Оцінка майна клієнта, дол",object


In [95]:
# Converting incompatible data types
df['Сума боргу, грн'] = df['Сума боргу, грн'].str.replace(',', '.', regex=False).astype('float')
df['Площа майна клієнта'] = df['Площа майна клієнта'].str.replace(',', '.', regex=False).astype('float')
df['Оцінка майна клієнта, дол'] = df['Оцінка майна клієнта, дол'].str.replace(' ', '', regex=False).astype('float')

In [96]:
# Creating missing columns with corresponding IDs and generating column content
df['Id_contract'] = np.random.randint(10000, 100000, size=len(df))
df['Id_client'] = np.random.randint(1000, 10000, size=len(df))
df['Id_property'] = np.random.randint(100, 1000, size=len(df))

In [98]:
df[['Surname', 'Name', 'SecondName']] = df['ПІБ'].str.split(' ', expand=True)

In [99]:
# Creating a DataFrame for each table
contracts = df[['Id_contract','Проект', 'Номер кредитного договору', 'Сума боргу, грн']].drop_duplicates()
contracts.columns = ['Id','ProjectName', 'ContractNum', 'Outstanding']

clients = df[['Id_client', 'Surname', 'Name', 'SecondName', 'ІПН']].drop_duplicates()
clients.columns = ['Id', 'Surname', 'Name', 'SecondName', 'IdentificationNumber']

properties = df[['Id_property','Регіон майна, що значиться за клієнтом', 'Тип майна клієнта', 'Площа майна клієнта', 'Оцінка майна клієнта, дол']].drop_duplicates()
properties.columns = ['Id','Region', 'PropertyType', 'PropertyArea', 'PropertyPrice']


In [101]:
# Assigning project and dataset IDs
project_id = 'database-project-431312'
dataset_id = 'clients'

In [111]:
# Checking and changing data types
contracts['ProjectName'] = contracts['ProjectName'].astype(str)
contracts['ContractNum'] = contracts['ContractNum'].astype(str)
contracts['Outstanding'] = pd.to_numeric(contracts['Outstanding'], errors='coerce')

clients['Surname'] = clients['Surname'].astype(str)
clients['Name'] = clients['Name'].astype(str)
clients['SecondName'] = clients['SecondName'].astype(str)
clients['IdentificationNumber'] = clients['IdentificationNumber'].astype(str)

properties['Region'] = properties['Region'].astype(str)
properties['PropertyType'] = properties['PropertyType'].astype(str)
properties['PropertyArea'] = pd.to_numeric(properties['PropertyArea'], errors='coerce')
properties['PropertyPrice'] = pd.to_numeric(properties['PropertyPrice'], errors='coerce')

# Filling NaN values
contracts = contracts.fillna(value=np.nan)
properties = properties.fillna(value=np.nan)

In [108]:
to_gbq(clients, f'{dataset_id}.Client', project_id=project_id, if_exists='append')

100%|██████████| 1/1 [00:00<00:00, 7639.90it/s]


In [109]:
to_gbq(contracts, f'{dataset_id}.Contract', project_id=project_id, if_exists='append')

100%|██████████| 1/1 [00:00<00:00, 1800.90it/s]


In [110]:
to_gbq(properties, f'{dataset_id}.Property', project_id=project_id, if_exists='append')

100%|██████████| 1/1 [00:00<00:00, 801.36it/s]
